In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np

def pca_reduction(indep_X, dep_Y, n_components, feature_names=None):

    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.2, random_state=42)

    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    explained_variance_ratio = pca.explained_variance_ratio_

    selected_features = None
    if feature_names is not None and isinstance(indep_X, (pd.DataFrame, np.ndarray)):
        try:
            components = pd.DataFrame(pca.components_, columns=feature_names)
            selected_features_indices = np.argsort(np.abs(components), axis=1)[:, ::-1]
            num_top_features = min(5, len(feature_names)) # Select top 5 contributing features per component
            top_features = []
            for i in range(n_components):
                top_indices = selected_features_indices[i, :num_top_features]
                top_features.append(components.columns[top_indices].tolist())
            selected_features = top_features
        except Exception as e:
            print(f"Error identifying contributing features: {e}")

    return X_train_pca, X_test_pca, y_train, y_test, explained_variance_ratio, selected_features

def split_scalar(indep_X_train, indep_X_test):
    sc = StandardScaler()
    X_train_scaled = sc.fit_transform(indep_X_train)
    X_test_scaled = sc.transform(indep_X_test)
    return X_train_scaled, X_test_scaled

def logistic(X_train, y_train, X_test, y_test):
    model = LogisticRegression(random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, accuracy, report, cm

def svm_linear(X_train, y_train, X_test, y_test):
    model = SVC(kernel='linear', random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, accuracy, report, cm

def svm_NL(X_train, y_train, X_test, y_test):
    model = SVC(kernel='rbf', random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, accuracy, report, cm

def knn(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, accuracy, report, cm

def navie(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, accuracy, report, cm

def Decision(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier(random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, accuracy, report, cm

def random(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, accuracy, report, cm

def pca_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    dataframe = pd.DataFrame(index=['PCA'], columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Navie', 'Decision', 'Random'])
    for number, idex in enumerate(dataframe.index):
        dataframe.loc[idex, 'Logistic'] = acclog[number]
        dataframe.loc[idex, 'SVMl'] = accsvml[number]
        dataframe.loc[idex, 'SVMnl'] = accsvmnl[number]
        dataframe.loc[idex, 'KNN'] = accknn[number]
        dataframe.loc[idex, 'Navie'] = accnav[number]
        dataframe.loc[idex, 'Decision'] = accdes[number]
        dataframe.loc[idex, 'Random'] = accrf[number]
    return dataframe

dataset1=pd.read_csv("Wine.csv",index_col=None)

df2=dataset1

df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('Customer_Segment',axis=1)
dep_Y=df2['Customer_Segment']
# Assuming you have already run your pca_reduction function and have indep_X and dep_Y
X_train_pca, X_test_pca, y_train_pca, y_test_pca, explained_variance, _ = pca_reduction(indep_X, dep_Y, 3, feature_names=indep_X.columns if isinstance(indep_X, pd.DataFrame) else None)

# Now scale the PCA-transformed data
X_train_scaled_pca, X_test_scaled_pca = split_scalar(X_train_pca, X_test_pca)

acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnav = []
accdes = []
accrf = []

classifier, Accuracy, report, cm = logistic(X_train_scaled_pca, y_train_pca, X_test_scaled_pca, y_test_pca)
acclog.append(Accuracy)

classifier, Accuracy, report, cm = svm_linear(X_train_scaled_pca, y_train_pca, X_test_scaled_pca, y_test_pca)
accsvml.append(Accuracy)

classifier, Accuracy, report, cm = svm_NL(X_train_scaled_pca, y_train_pca, X_test_scaled_pca, y_test_pca)
accsvmnl.append(Accuracy)

classifier, Accuracy, report, cm = knn(X_train_scaled_pca, y_train_pca, X_test_scaled_pca, y_test_pca)
accknn.append(Accuracy)

classifier, Accuracy, report, cm = navie(X_train_scaled_pca, y_train_pca, X_test_scaled_pca, y_test_pca)
accnav.append(Accuracy)

classifier, Accuracy, report, cm = Decision(X_train_scaled_pca, y_train_pca, X_test_scaled_pca, y_test_pca)
accdes.append(Accuracy)

classifier, Accuracy, report, cm = random(X_train_scaled_pca, y_train_pca, X_test_scaled_pca, y_test_pca)
accrf.append(Accuracy)

result = pca_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
print(result)

     Logistic      SVMl     SVMnl       KNN     Navie  Decision    Random
PCA  0.916667  0.916667  0.944444  0.888889  0.888889  0.888889  0.916667
